#  UNIVERS 3 – LES ADMINISTRATEURS (PRÊTE-NOMS)

In [50]:
print("🥇 UNIVERS 3 – ANALYSE DES ADMINISTRATEURS ET PRÊTE-NOMS")
print("=" * 60)


🥇 UNIVERS 3 – ANALYSE DES ADMINISTRATEURS ET PRÊTE-NOMS


## IMPORTS ET CONFIGURATION

In [51]:
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
import plotly.express as px
from neo4j import GraphDatabase, basic_auth
import numpy as np
from collections import Counter
from node2vec import Node2Vec
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

# Chemins des fichiers
nodes_path = r"C:\Users\Ashahi\Desktop\Graph\note3\subgraphs_narratifs\sousgraphe_3_officers_nodes.csv"
edges_path = r"C:\Users\Ashahi\Desktop\Graph\note3\subgraphs_narratifs\sousgraphe_3_officers_edges.csv"

# Configuration Neo4j
URI = "bolt://localhost:7687"
USER = "neo4j"
PASSWORD = "555555555"

driver = GraphDatabase.driver(URI, auth=basic_auth(USER, PASSWORD))
print("✅ Connexion Neo4j établie")

✅ Connexion Neo4j établie


## INGESTION DES DONNÉES DANS NEO4J 

In [52]:
def execute_cypher(tx, query, params=None):
    """Wrapper pour exécuter Cypher"""
    if params is None:
        result = tx.run(query)
    else:
        result = tx.run(query, **params)
    return [record for record in result]

def setup_neo4j_univers3(driver, nodes_path, edges_path):
    with driver.session() as session:
        # Nettoyage
        session.execute_write(execute_cypher, "MATCH (n) DETACH DELETE n")
        print("🧹 Base nettoyée")
        
        # Index
        session.execute_write(execute_cypher, "CREATE INDEX officer_id_index IF NOT EXISTS FOR (n:Officer) ON (n.node_id)")
        
        # Import des nœuds
        nodes_df = pd.read_csv(nodes_path, low_memory=False).fillna('')
        node_id_col = next((c for c in nodes_df.columns if c.lower() in ("id", "node_id", "node")), nodes_df.columns[0])
        node_params = nodes_df.rename(columns={node_id_col: 'node_id'}).to_dict('records')
        
        create_nodes_query = """
        UNWIND $props_list AS props
        MERGE (n:Officer {node_id: toString(props.node_id)})
        SET n += apoc.map.clean(props, ['node_id'], [''])
        """
        session.execute_write(execute_cypher, create_nodes_query, {"props_list": node_params})
        print(f"✅ {len(nodes_df)} nœuds administrateurs importés")
        
        # Import des arêtes
        edges_df = pd.read_csv(edges_path, low_memory=False).fillna('')
        src_col = next((c for c in edges_df.columns if c.lower() in ("source", "from", "node_id_start")), edges_df.columns[0])
        tgt_col = next((c for c in edges_df.columns if c.lower() in ("target", "to", "node_id_end")), edges_df.columns[1])
        edge_params = edges_df.rename(columns={src_col: 'source_id', tgt_col: 'target_id'}).to_dict('records')
        
        create_edges_query = """
        UNWIND $props_list AS props
        MATCH (a:Officer {node_id: toString(props.source_id)})
        MATCH (b:Officer {node_id: toString(props.target_id)})
        MERGE (a)-[r:RELATES_TO]->(b)
        SET r += apoc.map.clean(props, ['source_id', 'target_id'], [''])
        """
        session.execute_write(execute_cypher, create_edges_query, {"props_list": edge_params})
        print(f"✅ {len(edges_df)} relations administrateurs importées")

print("🚀 Ingestion des données administrateurs...")
setup_neo4j_univers3(driver, nodes_path, edges_path)

🚀 Ingestion des données administrateurs...
🧹 Base nettoyée
✅ 700 nœuds administrateurs importés
✅ 3279 relations administrateurs importées


## CALCUL DES CENTRALITÉS

In [53]:
def calculate_officer_centrality(driver):
    with driver.session() as session:
        # Supprimer graphe existant
        try:
            session.execute_write(execute_cypher, "CALL gds.graph.drop('officersGraph', false)")
        except:
            pass
        
        # Créer le graphe projeté
        session.execute_write(execute_cypher, """
        CALL gds.graph.project('officersGraph', 'Officer', 'RELATES_TO')
        """)
        print("✅ Graphe officersGraph projeté")
        
        # Degree Centrality
        session.execute_write(execute_cypher, """
        CALL gds.degree.write('officersGraph', {writeProperty: 'degree'})
        YIELD nodePropertiesWritten
        """)
        print("✅ Degree Centrality calculée")
        
        # Betweenness Centrality
        session.execute_write(execute_cypher, """
        CALL gds.betweenness.write('officersGraph', {
            writeProperty: 'betweenness', 
            concurrency: 4
        })
        YIELD nodePropertiesWritten
        """)
        print("✅ Betweenness Centrality calculée")
        
        # Nettoyage
        session.execute_write(execute_cypher, "CALL gds.graph.drop('officersGraph')")

print("🎯 Calcul des centralités des administrateurs...")
calculate_officer_centrality(driver)


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('schema' returned by 'gds.graph.drop' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL gds.graph.drop('officersGraph', false)"
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('schema' returned by 'gds.graph.drop' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL gds.graph.drop('officersGraph')"


🎯 Calcul des centralités des administrateurs...
✅ Graphe officersGraph projeté
✅ Degree Centrality calculée
✅ Betweenness Centrality calculée


## EXTRACTION ET INSPECTION DES DONNÉES

In [54]:
def extract_officers_data(driver):
    """Extraire les données administrateurs depuis Neo4j"""
    query = """
    MATCH (o:Officer)
    RETURN 
        toString(o.node_id) AS id,
        o.name AS name,
        o.node_type AS node_type,
        o.jurisdiction_description AS jurisdiction_description,
        o.countries AS countries,
        o.degree AS degree,
        o.betweenness AS betweenness
    ORDER BY o.betweenness DESC
    """
    
    with driver.session() as session:
        result = session.run(query)
        df = pd.DataFrame([dict(record) for record in result])
    
    return df

def extract_officers_graph(driver):
    """Extraire la structure complète du graphe"""
    nodes_query = """
    MATCH (o:Officer)
    RETURN toString(o.node_id) AS id, o.name AS name, o.degree AS degree, o.betweenness AS betweenness
    """
    
    edges_query = """
    MATCH (a:Officer)-[r:RELATES_TO]->(b:Officer)
    RETURN toString(a.node_id) AS source, toString(b.node_id) AS target
    """
    
    with driver.session() as session:
        nodes_df = pd.DataFrame([dict(record) for record in session.run(nodes_query)])
        edges_df = pd.DataFrame([dict(record) for record in session.run(edges_query)])
    
    return nodes_df, edges_df

print("🔍 Extraction des données administrateurs...")
officers_df = extract_officers_data(driver)
nodes_df, edges_df = extract_officers_graph(driver)

# Construction du graphe NetworkX
G_officers = nx.from_pandas_edgelist(edges_df, source='source', target='target')

# Ajout des attributs
nodes_dict = nodes_df.set_index('id').to_dict('index')
for node_id, attrs in nodes_dict.items():
    if node_id in G_officers.nodes():
        for attr_name, attr_value in attrs.items():
            G_officers.nodes[node_id][attr_name] = attr_value

print(f"✅ Graphe administrateurs construit: {G_officers.number_of_nodes()} nœuds, {G_officers.number_of_edges()} arêtes")

🔍 Extraction des données administrateurs...
✅ Graphe administrateurs construit: 676 nœuds, 3279 arêtes


## INSPECTION ET DIAGNOSTIC DES DONNÉES

In [55]:
print("\n🔍 DIAGNOSTIC DU RÉSEAU DES ADMINISTRATEURS")
print("=" * 50)

print(f"📊 STATISTIQUES DE BASE:")
print(f"• Administrateurs: {len(officers_df)}")
print(f"• Relations: {len(edges_df)}")
print(f"• Densité du réseau: {nx.density(G_officers):.6f}")


🔍 DIAGNOSTIC DU RÉSEAU DES ADMINISTRATEURS
📊 STATISTIQUES DE BASE:
• Administrateurs: 700
• Relations: 3279
• Densité du réseau: 0.014372


In [56]:
print(f"\n🏷️ TYPES DE NŒUDS:")
node_types = officers_df['node_type'].value_counts()
for typ, count in node_types.items():
    print(f"• {typ}: {count}")


🏷️ TYPES DE NŒUDS:
• Entity: 642
• Intermediary: 29
• Officer: 19
• Address: 10


In [57]:
print(f"\n🌍 DISTRIBUTION GÉOGRAPHIQUE (Top 10):")
jurisdictions = officers_df['jurisdiction_description'].value_counts().head(10)
for jur, count in jurisdictions.items():
    print(f"• {jur}: {count}")


🌍 DISTRIBUTION GÉOGRAPHIQUE (Top 10):
• British Virgin Islands: 633
• Cayman Islands: 8
• Samoa: 1


In [58]:
print(f"\n📈 CENTRALITÉ DES ADMINISTRATEURS:")
print(f"• Degré moyen: {officers_df['degree'].mean():.1f}")
print(f"• Degré max: {officers_df['degree'].max()}")
print(f"• Betweenness moyen: {officers_df['betweenness'].mean():.6f}")
print(f"• Betweenness max: {officers_df['betweenness'].max():.6f}")


📈 CENTRALITÉ DES ADMINISTRATEURS:
• Degré moyen: 4.7
• Degré max: 485.0
• Betweenness moyen: 220.642857
• Betweenness max: 48915.351490


In [59]:
# Visualisation de la distribution des degrés
fig = px.histogram(officers_df, x='degree', nbins=50, 
                   title='Distribution du Degré des Administrateurs',
                   labels={'degree': 'Nombre de connexions', 'count': 'Nombre d\'administrateurs'})
fig.show()



In [60]:
# ---------------------------
# Nettoyage & variables manquantes (patch)
# ---------------------------

# Assurer que les ids dans nodes/edges sont des chaînes (cohérence)
nodes_df['id'] = nodes_df['id'].astype(str)
edges_df['source'] = edges_df['source'].astype(str)
edges_df['target'] = edges_df['target'].astype(str)

# Nettoyage général et types
# Créer officers_df_clean pour usages ultérieurs (export, top, heatmap)
officers_df_clean = officers_df.copy()

# Remplacer NaN par valeurs par défaut
officers_df_clean['name'] = officers_df_clean['name'].fillna('')
# degree -> entier (si possible), sinon 0
officers_df_clean['degree'] = pd.to_numeric(officers_df_clean['degree'], errors='coerce').fillna(0).astype(int)
# betweenness -> float
officers_df_clean['betweenness'] = pd.to_numeric(officers_df_clean['betweenness'], errors='coerce').fillna(0.0).astype(float)
officers_df_clean['jurisdiction_description'] = officers_df_clean.get('jurisdiction_description', '').fillna('')

print("🧾 officers_df_clean créé: types standardisés")
print(officers_df_clean.dtypes)

# Définir top_officers (utilisé plusieurs fois)
TOP_K = 20
if 'betweenness' in officers_df_clean.columns:
    top_officers = officers_df_clean.nlargest(TOP_K, 'betweenness').reset_index(drop=True)
else:
    top_officers = officers_df_clean.head(TOP_K).reset_index(drop=True)
    print("⚠️ 'betweenness' absent — top_officers pris au premier ordre du DataFrame")

print(f"🏆 top_officers défini ({len(top_officers)} éléments) — Top {TOP_K} par betweenness")

# (Re)construire le graphe NetworkX de façon robuste avec des noeuds garantis en string
G_officers = nx.from_pandas_edgelist(edges_df, source='source', target='target', create_using=nx.Graph())

# Ajouter (ou mettre à jour) attributs depuis nodes_df / officers_df_clean
# Priorité: officers_df_clean (qui contient degree/betweenness normalisés)
nodes_dict = officers_df_clean.set_index('id').to_dict('index')
for node_id, attrs in nodes_dict.items():
    if node_id not in G_officers.nodes():
        # ajouter le noeud même s'il n'a pas d'edge — utile pour les administrateurs isolés
        G_officers.add_node(node_id)
    for attr_name, attr_value in attrs.items():
        G_officers.nodes[node_id][attr_name] = attr_value

print(f"✅ Graphe administrateurs construit: {G_officers.number_of_nodes()} nœuds, {G_officers.number_of_edges()} arêtes")

# Sécurité: si top_officers est utilisé plus bas et que certains ids ne figurent pas dans le graphe,
# on alerte mais on continue.
missing_in_graph = [nid for nid in top_officers['id'].astype(str).tolist() if nid not in G_officers.nodes()]
if missing_in_graph:
    print(f"⚠️ {len(missing_in_graph)} top_officers n'ont pas d'entrée dans G_officers (IDs retirés/absents) — exemples: {missing_in_graph[:5]}")


🧾 officers_df_clean créé: types standardisés
id                           object
name                         object
node_type                    object
jurisdiction_description     object
countries                    object
degree                        int32
betweenness                 float64
dtype: object
🏆 top_officers défini (20 éléments) — Top 20 par betweenness
✅ Graphe administrateurs construit: 700 nœuds, 3279 arêtes


In [61]:
# Version simplifiée et robuste pour l'affichage du Top 20
print(f"\n🏆 TOP 20 ADMINISTRATEURS LES PLUS CENTRAUX:")
print("-" * 80)

for i, row in top_officers.iterrows():
    name = str(row['name'])[:40] + "..." if len(str(row['name'])) > 40 else str(row['name'])
    degree = int(row['degree'])  # Conversion garantie en int
    betweenness = float(row['betweenness'])  # Conversion garantie en float
    
    # Formatage simple et robuste
    print(f"{i+1:2d}. {name:<45} Degré: {degree:3d} Betweenness: {betweenness:8.6f}")


🏆 TOP 20 ADMINISTRATEURS LES PLUS CENTRAUX:
--------------------------------------------------------------------------------
 1.                                               Degré: 248 Betweenness: 48915.351490
 2. Portcullis TrustNet (BVI) Limited             Degré: 185 Betweenness: 37828.359334
 3. Execorp Limited                               Degré: 485 Betweenness: 27506.798276
 4. UBS AG, Singapore                             Degré: 244 Betweenness: 17203.150000
 5. Sharecorp Limited                             Degré:  58 Betweenness: 14971.779160
 6. Acticorp Limited                              Degré:  39 Betweenness: 2009.942937
 7. Directcorp Limited                            Degré:   8 Betweenness: 611.472727
 8. FORTUNE SPREAD INVESTMENT LIMITED             Degré:   4 Betweenness: 397.000000
 9. SPITFIRE TRADING COMPANY LIMITED              Degré:   3 Betweenness: 386.588402
10. KIKI HOLDING (PTC) LIMITED                    Degré:   2 Betweenness: 348.451065
11. BLACK PEA

## NODE2VEC - EMBEDDINGS DES ADMINISTRATEURS

In [62]:
print("\n🔮 NODE2VEC - ANALYSE DES EMBEDDINGS")
print("=" * 40)

# Configuration Node2Vec
print("🎯 Entraînement des embeddings Node2Vec...")
node2vec = Node2Vec(G_officers, 
                    dimensions=64,      # Dimension des embeddings
                    walk_length=30,     # Longueur des walks
                    num_walks=200,      # Nombre de walks par nœud
                    workers=2,          # Parallelisation
                    p=1,                # Paramètre de retour
                    q=1)                # Paramètre de exploration

# Entraînement du modèle
model = node2vec.fit(window=10, min_count=1, batch_words=4)
print("✅ Modèle Node2Vec entraîné")

# Fonction pour obtenir les embeddings
def get_node_embeddings(model, graph):
    """Extraire les embeddings pour tous les nœuds"""
    embeddings = {}
    for node in graph.nodes():
        if str(node) in model.wv:
            embeddings[node] = model.wv[str(node)]
    return embeddings

# Extraction des embeddings
embeddings = get_node_embeddings(model, G_officers)
print(f"✅ Embeddings extraits pour {len(embeddings)} nœuds")


🔮 NODE2VEC - ANALYSE DES EMBEDDINGS
🎯 Entraînement des embeddings Node2Vec...


Computing transition probabilities: 100%|██████████| 700/700 [00:17<00:00, 40.02it/s]


✅ Modèle Node2Vec entraîné
✅ Embeddings extraits pour 700 nœuds


## ANALYSE DE SIMILARITÉ ET CLUSTERING

In [63]:
# ==========================================================
# 🎯 ANALYSE DES SIMILARITÉS ENTRE ADMINISTRATEURS (corrigée)
# ==========================================================

print("🎯 ANALYSE DES SIMILARITÉS ENTRE ADMINISTRATEURS")
print("==================================================")

# Exemple : top 3 administrateurs du modèle node2vec
for idx, row in top_officers.head(3).iterrows():
    officer_id = str(row['id'])
    officer_name = row.get('name', 'N/A')

    print(f"🔍 Administrateurs similaires à: {officer_name}")

    # Calcul des similarités à d'autres administrateurs
    similar_officers = []
    for other_id in G_officers.nodes():
        if other_id == officer_id:
            continue
        try:
            similarity = model.wv.similarity(officer_id, other_id)
        except KeyError:
            # si un noeud n'a pas d'embedding
            continue
        similar_officers.append((other_id, similarity))

    # Trier par similarité décroissante
    similar_officers = sorted(similar_officers, key=lambda x: x[1], reverse=True)[:3]

    # Afficher les top similaires avec infos
    for i, (other_id, similarity) in enumerate(similar_officers):
        officer_info = officers_df_clean[officers_df_clean['id'].astype(str) == str(other_id)]

        if officer_info.empty:
            print(f"  ⚠️ Aucune info trouvée pour l’ID {other_id} (probablement hors du top_officers)")
            continue

        name = officer_info.iloc[0].get('name', 'Inconnu')
        degree = officer_info.iloc[0].get('degree', 'N/A')

        print(f"  {i+1}. {name[:40]}... (similarité: {similarity:.3f}, degré: {degree})")

print("\n✅ Analyse de similarité terminée sans erreur.")


🎯 ANALYSE DES SIMILARITÉS ENTRE ADMINISTRATEURS
🔍 Administrateurs similaires à: 
  1. Portcullis TrustNet (BVI) Limited... (similarité: 0.972, degré: 185)
  2. Sharecorp Limited... (similarité: 0.882, degré: 58)
  3. ... (similarité: 0.763, degré: 0)
🔍 Administrateurs similaires à: Portcullis TrustNet (BVI) Limited
  1. ... (similarité: 0.972, degré: 248)
  2. Sharecorp Limited... (similarité: 0.884, degré: 58)
  3. Execorp Limited... (similarité: 0.774, degré: 485)
🔍 Administrateurs similaires à: Execorp Limited
  1. Sharecorp Limited... (similarité: 0.822, degré: 58)
  2. Portcullis TrustNet (BVI) Limited... (similarité: 0.774, degré: 185)
  3. ... (similarité: 0.761, degré: 248)

✅ Analyse de similarité terminée sans erreur.


## VISUALISATION T-SNE DES EMBEDDINGS

In [64]:
print("\n🎨 VISUALISATION T-SNE DES EMBEDDINGS")
print("=" * 40)

if len(embeddings) > 10:
    # Réduction de dimension avec t-SNE
    tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, len(embeddings)-1))
    embeddings_2d = tsne.fit_transform(embedding_matrix)
    
    # Création du DataFrame pour visualisation
    viz_df = pd.DataFrame({
        'x': embeddings_2d[:, 0],
        'y': embeddings_2d[:, 1],
        'cluster': clusters,
        'node_id': node_ids
    })
    
    # Fusion avec les données des administrateurs
    viz_df = viz_df.merge(officers_df[['id', 'name', 'degree', 'betweenness']], 
                         left_on='node_id', right_on='id', how='left')
    
    # Visualisation Plotly
    fig = px.scatter(viz_df, x='x', y='y', color='cluster',
                     hover_data=['name', 'degree', 'betweenness'],
                     title='Visualisation T-SNE des Administrateurs par Similarité Structurelle',
                     labels={'cluster': 'Cluster'})
    
    fig.update_traces(marker=dict(size=8, line=dict(width=1, color='DarkSlateGrey')),
                      selector=dict(mode='markers'))
    
    fig.show()


🎨 VISUALISATION T-SNE DES EMBEDDINGS


In [65]:
fig.write_html(
    r"C:\Users\Ashahi\Desktop\Graph\examen\notebooks\graphes_notebook2\graphe_1.html",
    include_plotlyjs='cdn',
    full_html=True
)

Cette visualisation T-SNE montre que les administrateurs se regroupent en **plusieurs clusters distincts**, indiquant des **profils structurels similaires à l’intérieur de chaque groupe** (rôles, positions ou connexions comparables dans le réseau), tandis que la **séparation nette entre les clusters** traduit des **différences marquées de structure ou de fonction** entre ces groupes.


## VISUALISATION DU RÉSEAU DES ADMINISTRATEURS

In [66]:
print("\n🔥 MATRICE DES CONNEXIONS ENTRE CLUSTERS")
print("=" * 45)

# =====================================================
# 1️⃣ CALCUL DE LA MATRICE DE CONNEXION
# =====================================================

# S'assurer que viz_df existe et a les clusters
if 'viz_df' not in locals():
    print("❌ viz_df non trouvé, création depuis les embeddings...")
    # Recréer viz_df si nécessaire (code de votre T-SNE)
    embedding_matrix = np.array(list(embeddings.values()))
    node_ids = list(embeddings.keys())
    
    tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, len(embeddings)-1))
    embeddings_2d = tsne.fit_transform(embedding_matrix)
    
    kmeans = KMeans(n_clusters=5, random_state=42)
    clusters = kmeans.fit_predict(embedding_matrix)
    
    viz_df = pd.DataFrame({
        'x': embeddings_2d[:, 0],
        'y': embeddings_2d[:, 1],
        'cluster': clusters,
        'node_id': node_ids
    }).merge(officers_df[['id', 'name', 'degree', 'betweenness']], 
             left_on='node_id', right_on='id', how='left')

# Obtenir la liste des clusters
cluster_ids = sorted(viz_df['cluster'].unique())
print(f"🔍 Analyse de {len(cluster_ids)} clusters")

# Initialiser la matrice de connexion
connection_matrix = np.zeros((len(cluster_ids), len(cluster_ids)))

# Compter les connexions entre clusters
total_connections = 0
inter_cluster_connections = 0

for edge in G_officers.edges():
    node1, node2 = edge
    
    # Vérifier si les deux nœuds sont dans viz_df
    node1_in_viz = node1 in viz_df['node_id'].values
    node2_in_viz = node2 in viz_df['node_id'].values
    
    if node1_in_viz and node2_in_viz:
        cluster1 = viz_df[viz_df['node_id'] == node1]['cluster'].iloc[0]
        cluster2 = viz_df[viz_df['node_id'] == node2]['cluster'].iloc[0]
        
        i = cluster_ids.index(cluster1)
        j = cluster_ids.index(cluster2)
        
        connection_matrix[i][j] += 1
        total_connections += 1
        
        if cluster1 != cluster2:
            inter_cluster_connections += 1

print(f"✅ {total_connections} connexions analysées")
print(f"🔗 {inter_cluster_connections} connexions inter-clusters ({inter_cluster_connections/max(1, total_connections)*100:.1f}%)")



🔥 MATRICE DES CONNEXIONS ENTRE CLUSTERS
🔍 Analyse de 5 clusters
✅ 3279 connexions analysées
🔗 972 connexions inter-clusters (29.6%)


In [67]:
# =====================================================
# 2️⃣ CRÉATION DE LA MATRICE SYMÉTRIQUE
# =====================================================

# Rendre la matrice symétrique pour une meilleure visualisation
symmetric_matrix = connection_matrix + connection_matrix.T
np.fill_diagonal(symmetric_matrix, symmetric_matrix.diagonal() / 2)  # Éviter le double comptage

In [68]:
# =====================================================
# 3️⃣ VISUALISATION HEATMAP AVANCÉE
# =====================================================

fig_heatmap = go.Figure(data=go.Heatmap(
    z=symmetric_matrix,
    x=[f'Cluster {c}' for c in cluster_ids],
    y=[f'Cluster {c}' for c in cluster_ids],
    colorscale='Blues',
    hoverongaps=False,
    text=symmetric_matrix.astype(int),
    texttemplate="%{text}",
    textfont={"size": 14, "color": "black"},
    hovertemplate=(
        "De: <b>%{y}</b><br>"
        "Vers: <b>%{x}</b><br>"
        "Connexions: <b>%{z}</b><br>"
        "<extra></extra>"
    )
))

fig_heatmap.update_layout(
    title="🔗 MATRICE DES CONNEXIONS ENTRE CLUSTERS<br>"
          "<span style='font-size:12px; color:#666'>Nombre de liens entre les clusters d'administrateurs</span>",
    width=700,
    height=650,
    xaxis=dict(title="Cluster cible"),
    yaxis=dict(title="Cluster source"),
    font=dict(size=12)
)

fig_heatmap.show()


In [69]:
fig.write_html(
    r"C:\Users\Ashahi\Desktop\Graph\examen\notebooks\graphes_notebook2\graphe_2.html",
    include_plotlyjs='cdn',
    full_html=True
)

In [70]:
# =====================================================
# 4️⃣ ANALYSE DES RÉSULTATS
# =====================================================

print(f"\n📊 ANALYSE DE LA MATRICE DE CONNEXION:")

# Cluster le plus connecté vers l'extérieur
outgoing_connections = symmetric_matrix.sum(axis=1) - np.diag(symmetric_matrix)
most_connected_out = cluster_ids[np.argmax(outgoing_connections)]
print(f"• Cluster le plus connecté vers l'extérieur: Cluster {most_connected_out} ({outgoing_connections.max():.0f} liens)")

# Cluster le plus connecté depuis l'extérieur  
incoming_connections = symmetric_matrix.sum(axis=0) - np.diag(symmetric_matrix)
most_connected_in = cluster_ids[np.argmax(incoming_connections)]
print(f"• Cluster le plus connecté depuis l'extérieur: Cluster {most_connected_in} ({incoming_connections.max():.0f} liens)")

# Densité des connexions inter-clusters
total_possible_inter = len(cluster_ids) * (len(cluster_ids) - 1)
actual_inter = np.sum(symmetric_matrix) - np.sum(np.diag(symmetric_matrix))
inter_cluster_density = actual_inter / total_possible_inter if total_possible_inter > 0 else 0
print(f"• Densité des connexions inter-clusters: {inter_cluster_density:.3f}")


📊 ANALYSE DE LA MATRICE DE CONNEXION:
• Cluster le plus connecté vers l'extérieur: Cluster 3 (958 liens)
• Cluster le plus connecté depuis l'extérieur: Cluster 3 (958 liens)
• Densité des connexions inter-clusters: 97.200


In [71]:
# =====================================================
# 5️⃣ VISUALISATION COMPLÉMENTAIRE - GRAPHE DES CONNEXIONS
# =====================================================

print(f"\n🔄 GRAPHE DES CONNEXIONS ENTRE CLUSTERS:")
print("=" * 40)

# Créer un graphe où les nœuds sont les clusters
G_cluster = nx.Graph()

# Ajouter les clusters comme nœuds
for cluster in cluster_ids:
    cluster_size = len(viz_df[viz_df['cluster'] == cluster])
    G_cluster.add_node(cluster, size=cluster_size)

# Ajouter les arêtes entre clusters (poids = nombre de connexions)
for i, cluster1 in enumerate(cluster_ids):
    for j, cluster2 in enumerate(cluster_ids):
        if i < j and symmetric_matrix[i][j] > 0:  # Éviter les doublons
            G_cluster.add_edge(cluster1, cluster2, weight=symmetric_matrix[i][j])

# Visualisation du graphe des clusters
pos_cluster = nx.spring_layout(G_cluster, seed=42)

fig_cluster_network = go.Figure()

# Arêtes entre clusters
edge_x, edge_y = [], []
for edge in G_cluster.edges():
    x0, y0 = pos_cluster[edge[0]]
    x1, y1 = pos_cluster[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])
    
    # Ajouter une trace pour le poids de l'arête
    fig_cluster_network.add_trace(go.Scatter(
        x=[(x0+x1)/2], y=[(y0+y1)/2],
        mode='text',
        text=[f"{G_cluster.edges[edge]['weight']:.0f}"],
        textfont=dict(size=12, color='red'),
        showlegend=False,
        hoverinfo='none'
    ))

fig_cluster_network.add_trace(go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=2, color='gray'),
    hoverinfo='none',
    mode='lines',
    showlegend=False
))

# Nœuds (clusters)
node_x, node_y, node_sizes, node_text = [], [], [], []
for cluster in cluster_ids:
    x, y = pos_cluster[cluster]
    node_x.append(x)
    node_y.append(y)
    
    cluster_size = len(viz_df[viz_df['cluster'] == cluster])
    node_sizes.append(20 + cluster_size * 2)
    
    node_text.append(f"Cluster {cluster}<br>{cluster_size} membres")

fig_cluster_network.add_trace(go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    marker=dict(
        size=node_sizes,
        color=cluster_ids,
        colorscale='Viridis',
        line=dict(width=2, color='white')
    ),
    text=[f"Cluster {c}" for c in cluster_ids],
    textposition="middle center",
    hovertext=node_text,
    hoverinfo='text',
    showlegend=False
))

fig_cluster_network.update_layout(
    title="🔄 RÉSEAU DES CONNEXIONS ENTRE CLUSTERS<br>"
          "<span style='font-size:12px; color:#666'>Taille = Nombre de membres • Texte rouge = Nombre de liens</span>",
    width=800,
    height=600,
    showlegend=False,
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
)

fig_cluster_network.show()


🔄 GRAPHE DES CONNEXIONS ENTRE CLUSTERS:


In [72]:
fig.write_html(
    r"C:\Users\Ashahi\Desktop\Graph\examen\notebooks\graphes_notebook2\graphe_3.html",
    include_plotlyjs='cdn',
    full_html=True
)

In [73]:
print("\n🌐 VISUALISATION PAR CENTRALITÉ (sans clustering)")
print("=" * 45)

# Prendre moins de nœuds pour mieux voir (30 au lieu de 100)
top_officer_ids = set(officers_df.nlargest(30, 'betweenness')['id'])
G_top_officers = G_officers.subgraph(top_officer_ids).copy()

print(f"• Réseau visualisé: {len(G_top_officers)} administrateurs")
print(f"• Connexions: {G_top_officers.number_of_edges()}")

# Layout avec plus d'espace
pos = nx.spring_layout(G_top_officers, k=5, iterations=100, seed=42)  # k=5 pour plus d'espace

# Arêtes
edge_x, edge_y = [], []
for edge in G_top_officers.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=1, color='rgba(150, 150, 150, 0.4)'),
    hoverinfo='none',
    mode='lines',
    showlegend=False
)

# Nœuds avec couleur par betweenness (au lieu de cluster)
node_x, node_y, colors, sizes, hover_texts = [], [], [], [], []

for node in G_top_officers.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    
    # Données de l'administrateur
    officer_data = officers_df[officers_df['id'] == node].iloc[0]
    
    # Couleur par betweenness (plus intéressant qu'un seul cluster)
    betweenness = officer_data['betweenness']
    colors.append(betweenness)
    
    # Taille par degree
    degree = officer_data['degree']
    sizes.append(15 + np.log1p(degree) * 5)  # Scaling logarithmique
    
    # Texte hover
    hover_text = (
        f"<b>{officer_data['name']}</b><br>"
        f"Degré: {degree}<br>"
        f"Betweenness: {betweenness:.4f}<br>"
        f"Pays: {officer_data.get('jurisdiction_description', 'N/A')}"
    )
    hover_texts.append(hover_text)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    marker=dict(
        size=sizes,
        color=colors,
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title="Betweenness"),
        line=dict(width=2, color='white')
    ),
    text=hover_texts,
    hoverinfo='text',
    hovertemplate='%{text}<extra></extra>'
)

fig_network = go.Figure(data=[edge_trace, node_trace])
fig_network.update_layout(
    title="🥇 TOP 30 ADMINISTRATEURS PAR CENTRALITÉ<br>"
          "<span style='font-size:12px; color:#666'>Couleur = Betweenness • Taille = Degré</span>",
    width=1000,
    height=800,
    showlegend=False,
    hovermode='closest'
)

fig_network.show()


🌐 VISUALISATION PAR CENTRALITÉ (sans clustering)
• Réseau visualisé: 30 administrateurs
• Connexions: 84


In [74]:
fig.write_html(
    r"C:\Users\Ashahi\Desktop\Graph\examen\notebooks\graphes_notebook2\graphe_4.html",
    include_plotlyjs='cdn',
    full_html=True
)

In [75]:
print("\n🌐 VISUALISATION AVEC CLUSTERS MANUELS")
print("=" * 40)

# Créer des clusters simples basés sur le degree
officers_df['simple_cluster'] = pd.cut(officers_df['degree'], 
                                      bins=3, 
                                      labels=[0, 1, 2])

top_officer_ids = set(officers_df.nlargest(30, 'betweenness')['id'])
G_top_officers = G_officers.subgraph(top_officer_ids).copy()

# Layout
pos = nx.spring_layout(G_top_officers, k=5, iterations=100, seed=42)

# Même code pour les arêtes...

# Nœuds avec clusters manuels
node_x, node_y, colors, sizes, hover_texts = [], [], [], [], []

for node in G_top_officers.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    
    officer_data = officers_df[officers_df['id'] == node].iloc[0]
    
    # Couleur par cluster manuel
    cluster = officer_data['simple_cluster']
    colors.append(cluster)
    
    betweenness = officer_data['betweenness']
    sizes.append(15 + betweenness * 800)
    
    hover_text = (
        f"<b>{officer_data['name']}</b><br>"
        f"Cluster (par degré): {cluster}<br>"
        f"Degré: {officer_data['degree']}<br>"
        f"Betweenness: {betweenness:.4f}"
    )
    hover_texts.append(hover_text)

# Reste du code identique...


🌐 VISUALISATION AVEC CLUSTERS MANUELS


In [76]:
print("\n🔥 CARTE THERMIQUE DES SIMILARITÉS ENTRE ADMINISTRATEURS")
print("=" * 55)

# Matrice de similarité pour les top 15 administrateurs
top_15_ids = officers_df_clean.nlargest(15, 'betweenness')['id'].tolist()
similarity_matrix = np.zeros((len(top_15_ids), len(top_15_ids)))

# Remplir la matrice de similarité
for i, node1 in enumerate(top_15_ids):
    for j, node2 in enumerate(top_15_ids):
        if i == j:
            similarity_matrix[i][j] = 1.0
        elif str(node1) in model.wv and str(node2) in model.wv:
            similarity_matrix[i][j] = model.wv.similarity(str(node1), str(node2))
        else:
            similarity_matrix[i][j] = 0.0

# Noms pour les axes
names = []
for node_id in top_15_ids:
    name = officers_df_clean[officers_df_clean['id'] == node_id]['name'].iloc[0]
    names.append(str(name)[:20] + "..." if len(str(name)) > 20 else str(name))

# Carte thermique
fig_heatmap = go.Figure(data=go.Heatmap(
    z=similarity_matrix,
    x=names,
    y=names,
    colorscale='Viridis',
    hoverongaps=False,
    hovertemplate='Similarité entre<br>%{y} et<br>%{x}: %{z:.3f}<extra></extra>'
))

fig_heatmap.update_layout(
    title="🔥 SIMILARITÉS ENTRE LES TOP 15 ADMINISTRATEURS<br>"
          "<span style='font-size:12px; color:#666'>Basé sur les embeddings Node2Vec</span>",
    width=800,
    height=800,
    xaxis=dict(tickangle=45),
    yaxis=dict(tickangle=0)
)

fig_heatmap.show()


🔥 CARTE THERMIQUE DES SIMILARITÉS ENTRE ADMINISTRATEURS


In [77]:
fig.write_html(
    r"C:\Users\Ashahi\Desktop\Graph\examen\notebooks\graphes_notebook2\graphe_5.html",
    include_plotlyjs='cdn',
    full_html=True
)